# Introduction
This text gives an introduction to the undulator code in this package, with a concentration on the physics/maths implemented within it rather than instructions on how to use the code.  Such instructions will be documented later.

# ElectronBeam
The electron-beam is represented by a dictionary.  For example,
```
ebeam = {
    'energy': 3.0e9,
    'emitx': 340e-12,
    'emity': 8e-12,
    'betax': 9,
    'betay': 4.7,
    'espread': 0.8e-3,
}
```
Since this is Python, not all of these quantities will need to be defined for any particular function to work.  For example, the `beamgamma` function only needs the `'energy'` field to be present, and none of the others.  If you browse the code found in the `ebeam.py` module, you can see that many of the functions can be turned into simple look-ups by adding extra fields to the dictionary.

The `ebeam.py` module assumes that the beam has a waist at the centre of the undulator, and that the dispersion has been nulled for this section of the accelerator, and so the 4D transverse spot size can be calculated directly.

$$\sigma_{x,y}=\sqrt{\epsilon_{x,y}\beta_{x,y}}$$
$$\sigma'_{x,y}=\sqrt{\frac{\epsilon_{x,y}}{\beta_{x,y}}}$$

# InsertionDevice
The representation of the insertion device is also a simple Python dictionary.  For example,
```
ins_dev = {
    'Kmax': 1.38,
    'lamda_w': 18e-3,
    'Np': 111,
    'L': 2,
}
```

# Undulator.py
This is the module containing the vast majority of the physics complexity.

## Spectral properties
The wavelength, $\lambda_n$, of a particular harmonic, $n$, when viewed with an angle, $\theta$, with respect to the forward direction is calculated as follows.

$$
\lambda_n = \frac{\lambda_w}{2n\gamma_0^2}\left(1 + K_{max}^2 + \gamma_0^2\theta^2\right)
$$

From this, the photo energy, $E_\gamma$, can be calculated (where $h$, $c$, and $e$, are the usual physical constants).

$$
E_\gamma = \frac{hc}{e}\frac{1}{\lambda_n}
$$

There are two primary contributors to the width of each spectral line: the finite number of undulator periods, and the spread of the beam parameters that appear in that expression (i.e., the electron energy and divergence).

### Beam contribution
The width, $\sigma_{\lambda_{n,b}}$, due to the electron beam parameter spreads is calculated from a Taylor expansion of the $\lambda_n$ expression.

$$\Delta\lambda_n = \frac{\partial\lambda_n}{\partial \gamma} \Bigr|_{\substack{\gamma=\gamma_0}} \cdot \Delta\gamma + \frac{\partial\lambda_n}{\partial\Theta}\Bigr|_{\substack{\Theta=0}}\cdot\Delta\Theta + \frac{1}{2}\frac{\partial^2\lambda_n}{\partial\Theta^2}\Bigr|_{\substack{\Theta=0}}\cdot\Delta\Theta^2
$$

$$
\frac{\partial\lambda_n}{\partial \gamma}\Bigr|_{\substack{\gamma=\gamma_0}} = -\frac{1}{n\gamma_0^3}\left(1+K_w^2\right)\lambda_w
$$

$$
\frac{\partial\lambda_n}{\partial\Theta}\Bigr|_{\substack{\Theta=0}}=\frac{\Theta\lambda_w}{n}\Bigr|_{\substack{\Theta=0}} = 0
$$

$$
\frac{\partial^2\lambda_n}{\partial\Theta^2}\Bigr|_{\substack{\Theta=0}}=\frac{\lambda_w}{n}
$$

To calculate the RMS spread, it is assumed that the distributions of the energy-spread and the angular divergence are uncorrelated, and so can be added in quadrature.

$$
\sigma_{\lambda_{n,b}}^2 = \left(-\frac{1}{n}\frac{1}{\gamma_0^3}\left(1+K_w^2\right)\lambda_w\cdot\Delta\gamma\right)^2 + \left(\frac{1}{2}\frac{1}{n}\lambda_w\cdot\Delta\Theta^2\right)^2
$$

### Undulator contribution
The energy spectrum, $\frac{dW_n}{d\omega}$, of the $n$-th harmonic goes as follows.

$$
\frac{dW_n}{d\omega} \propto \left(\frac{\sin\left(\pi N_p \frac{\Delta\omega_n}{\omega_1}\right)}{\pi N_p \frac{\Delta\omega_n}{\omega_n}}\right)^2
$$

In many texts, the width of the spectrum due to the $\frac{\sin\left(Nx\right)}{Nx}$ is often approximated as the point at which this function first passes through zero.  This solution results in, $\frac{\Delta\omega_n}{\omega_n}=\pm\frac{1}{nN_p}$.  Unfortunately this does not match well with the RMS widths used elsewhere in the code, and so a different point is chosen.

$$
\frac{\sin\left(Nx\right)}{Nx} = \sqrt{\frac{1}{2}} \Longrightarrow \frac{\Delta\omega_n}{\omega_n}=\frac{0.225079}{N_p}
$$

This results in a spectral width, $\sigma_{\lambda_{n,u}}$, due to the finite number of periods in the undulator of,
$$
\sigma_{\lambda_{n,u}} = 0.225079\frac{1}{nN_p}\lambda_n
$$

### Total spectral width
The beam contribution and the undulator contribution should be convoluted to optain the total width, however it is enough to add the contributions in quadrature.

$$
\sigma_{\lambda_{n,total}}^2 = \left(-\frac{1}{n}\frac{1}{\gamma_0^3}\left(1+K_w^2\right)\lambda_w\cdot\Delta\gamma\right)^2 + \left(\frac{1}{2}\frac{1}{n}\lambda_w\cdot\Delta\Theta^2\right)^2 + \left(0.225079\frac{1}{nN_p}\lambda_n\right)^2
$$

## Example of a calculation of spectral properties
In the following I will give an example of how a calculation of the spectral properties of the light generated by an undulator could be calculated.  The values chosen are those of the NanoMAX beamline at MAX-IV.

First import the necessary components.

In [6]:
#from undulator.ebeam import sigy, sigyp, sigx, sigxp
from undulator.undulator import Undulator

Then create a beam (as a dictionary), and perform a few calculations.

In [3]:
ebeam = {
    'energy': 3e9,
    'emitx': 340e-12,
    'emity': 8e-12,
    'betax': 9,
    'betay': 4.7,
    'espread': 0.8e-3,
}
print(f'Vertical size RMS = {sigy(ebeam)*1e6:0.3f} um')
print(f'Vertical divergence RMS = {sigyp(ebeam)*1e6:0.3f} urad')
print(f'Horizontal size RMS = {sigx(ebeam)*1e6:0.3f} um')
print(f'Horizontal divergence RMS = {sigxp(ebeam)*1e6:0.3f} urad')

Vertical size RMS = 6.132 um
Vertical divergence RMS = 1.305 urad
Horizontal size RMS = 55.317 um
Horizontal divergence RMS = 6.146 urad


Now create a dictionary to represent the insertion device, and calculate its first harmonic (wavelength and photon energy).

In [5]:
K = 1.38
lamda_w = 18e-3
E = 3e9

ins_dev = {
    'Kmax': K,
    'lamda_w': lamda_w,
    'Np': 111,
    'L': 2,
}

n=1
wavel = phys.lamda_n(ins_dev, ebeam)
photonE = phys.energy_n(ins_dev, ebeam)
print(f'Central wavelength = {1e9 * wavel:0.30f} nm')
print(f'Corresponds to a photon energy of {1e-3 * photonE:0.3f} keV')

AttributeError: module 'undulator.undulator' has no attribute 'lamda_n'

Remember that the spectral width of the light will have a component that depends on the number of periods in the undulator and a component that depends on the spread of the energy and the divergence of the electron beam.  As well as calculating the total spectral width, the individual components can also be determined.

In [8]:
print(f'{phys.spectralwidth_ebeam(ins_dev, ebeam):0.3e}')
print(f'{phys.spectralwidth_undulator(ins_dev, ebeam):0.3e}')
print(f'{phys.spectralwidth_total(ins_dev, ebeam):0.3e}')

1.214e-12
1.538e-12
1.959e-12


## Transverse Size
The transverse size of the apparent source of the undulator light is a combination of the diffraction limited spot size and the transverse size fna divergence of the electron beam as it transits the undulator.

### Diffraction limited source size
The diffraction limited spot size, $\sigma_{diff}$, can be shown to be the following.
$$
\sigma_{diff} = \frac{1}{2\pi}\sqrt{\frac{1}{2}\lambda_nL}
$$

With the diffraction limited divergence, $\sigma'_{diff}$, being.
$$
\sigma'_{diff} = \frac{1}{2\pi}\sqrt{\frac{1}{2}\frac{\lambda_n}{L}}
$$

### Total source size
The total source size is the quadrature sum of several components:
1. The diffraction limited source size, $\sigma_{diff}$, as given above.
2. The RMS size, $\sigma_{b:x,y,x',y'}$, of the electron bunch in that plane of motion.
3. The average change in size of the bunch (in that plane of motion) as it traverses the undulator.  Note this is only true with the assumption that the electron beam optics lead to a waist within the undulator.
4. The apparent increase in size due to the oscillating path of the bunch within the undulator.

Note that the divergence of the electron beam is not affected by the undulator, and so only the first two points apply to the apparent source divergence.

$$
\sigma_{x,y}^2 = \sigma_{diff}^2 +  \sigma_{b:x,y}^2 + \frac{1}{3}\left(\frac{1}{2}\sigma_{x',y'}L\right)^2 + \left(\frac{\lambda_wK_{max}}{2\pi\gamma_0}\right)^2
$$

$$
\sigma_{x',y'}^2 = \sigma_{diff}'^2 +  \sigma_{b:x',y'}^2
$$

## Example of a calculation of the apparent source size
Blah blah

## Brightness
The brightness, $B$, of a photon beam from an undulator is defined as being inversely proportional to the 4D apparent source volume and the energy spread of the photons.

It is also proportional to the electron beam current, $I_b4.

$$
B \propto \frac{I_b}{\sigma_x\sigma_x'\sigma_y\sigma_y'\sigma_{\lambda_{n,total}}}
$$